# Organizing and Enriching Data Entities

Let's say you have resolved some data entities and stored them in a knowledge graph, but now it's time to organize them.

Often, this is done ad hoc following a hierarchical schema, like a taxonomy.

## Structuring Metadata in Graphs

Let's consider a couple aspects of knowledge graph development by example.

### Example: Organize your music taxonomically

In general, consider the familiar "schema" of popular music. In pseudo-Cypher, we can represent it like this

    (Artist)-[:RELEASED]->(Album)-[:INCLUDES]->(Song)

Where the nodes are modeled according to node Labels (Artist, Album, Song) and relationship Types (RELEASED, INCLUDES).

Each label might have some common properties that could be indexed on, for example:

    (Artist {knownAs, stageNames, nickNames, performerName})
    (Album {name, releaseDate, recordLabel, runtime})
    (Song {name, releaseDate, artist, producer, details})

Once organized in this kind of schema, it's easy to answer questions with queries "semantically", e.g.,
- *List all albums or songs released by a specific artist or group of artists*
- *Find all artists who have collaborated with each*
- *List all artists or albums containing a song with a word or phrase in its name*

### Example: Link additional nodes to music taxa

What's more - this schema can be built out more as needed. For example, I might want to group songs by Key or by Lyrics or Versions. The way this is done is a design choice. Let me give two examples...

In one case, let's say we know we often look for songs in the same key, so we decide that it should be a built-in property of a Song, so we might use Cypher to add that property:

    MERGE (s:Song {name: $name, artist: $artist, releaseDate: $releaseDate})
    SET s += {key: $key}

If we query this enough, then an alternative approach might be to create a node for Key

    (Key {name: $name, rootNote: $root_note, mode: $mode})<-[IN]-(Song)

Both approaches have their merit! We do not have hard rules about this.

## Demo: Research Data

The rest of this notebook focuses on research data. Our approach starts by scanning a filesystem and resolving all of the Folders as nodes based on information found in the NCDU scan. Note: Files can be included if the user chooses and does not exhaust the system).

**Goal:** *Create data entity models (aka "labels" or "classes"), with properties that represent the concept of data we are working on.*

Since we are working with `Image` data that was collected following the user's concept of cohort design, we are left with her categorical approach. In this case, she generally had three levels of folders where she stored the data. Each subject imaged is a `Mouse`.

The user provided an (understandably) messy version of the following details for each based on their workflow during acquisition:
- `Study`: e.g. U37, U19; these correspond with planned efforts to collect images
- `Date`: e.g. 211010, 20230221, 20Feb2022; these are handwritten dates of acquisition (there is actual metadata in the image headers that we can scrape)
- `Cage`: e.g. Cage1, Cage2, etc; these correspond with the animal housing, which was broken up into groups
- `CageID`: e.g NT, 1L, 2R; subjects were housed together and distinguished by ear tags that could be identified from these

Wherever we find unique (`Study`, `Cage`, `CageID`), we effectively have a unique `Mouse`, so we can actually continue adding entities. Given these details, we might create a schema like this

    (Study)-[INCLUDED]->(Cage)-[HOUSED]->(Mouse)-[REPRESENTED_IN]->(Image)-[STORED_IN]->(Folder)

linking Mouse, Image, and Folder location! Now, we have something really powerful we can attach to our data management graph.

Once we have stored `Image` and `Mouse` nodes in the system, we can pull a table of either on the fly, just like a relational database. Follow along below to see how this is done.

### Working with node data

In the following cell, load the csv that you've exported from the "Index" page of the app.

Once loaded, you can process data in a few steps:
- rename columns to work better with Neo4j
- add columns to describe the new nodes



In [2]:
import pandas as pd
from pathlib import Path
from datetime import datetime

input_fname = '../LauraM_USGI_uCT_Sessions.csv'
df = pd.read_csv(input_fname)

df.rename(columns={
    'Filepath': 'filepath',
    'Individual': 'CageID',
    'Date': 'date'
}, inplace=True)

df['Label'] = 'Image'
df['Instrument'] = 'microCT'
df['uid'] = df[['Study','Cage','CageID']].apply(lambda row: '-'.join([str(_i) for _i in row]), axis=1)

output_fname = f"{Path(input_fname).stem}_{datetime.today().ctime().replace(' ','_')}{Path(input_fname).suffix}"


base = '/mnt/server/bmc-lab6/atwai/archive/LaraM_Full_Dataset'
df['filepath'] = df['filepath'].apply(lambda x: f"{base}/{str(x)}")

df.to_csv(output_fname)
df

,filepath,Study,Cage,CageID,date,Label,Instrument,uid
0,/mnt/server/bmc-lab6/atwai/archive/LaraM_Full_...,U37,Cage006,1L,2022/10/21,Image,microCT,U37-Cage006-1L
1,/mnt/server/bmc-lab6/atwai/archive/LaraM_Full_...,U37,Cage006,1R,2022/10/21,Image,microCT,U37-Cage006-1R
2,/mnt/server/bmc-lab6/atwai/archive/LaraM_Full_...,U37,Cage008,NP,2022/10/21,Image,microCT,U37-Cage008-NP
3,/mnt/server/bmc-lab6/atwai/archive/LaraM_Full_...,U37,Cage008,1L,2022/10/21,Image,microCT,U37-Cage008-1L
4,/mnt/server/bmc-lab6/atwai/archive/LaraM_Full_...,U37,Cage008,1R,2022/10/21,Image,microCT,U37-Cage008-1R
...,...,...,...,...,...,...,...,...
2760,/mnt/server/bmc-lab6/atwai/archive/LaraM_Full_...,TBD,Cage090,2660,2022/02/03,Image,microCT,TBD-Cage090-2660
2761,/mnt/server/bmc-lab6/atwai/archive/LaraM_Full_...,TBD,Cage090,2674,2022/02/03,Image,microCT,TBD-Cage090-2674
2762,/mnt/server/bmc-lab6/atwai/archive/LaraM_Full_...,TBD,Cage112,2933,2022/02/03,Image,microCT,TBD-Cage112-2933
2763,/mnt/server/bmc-lab6/atwai/archive/LaraM_Full_...,TBD,Cage112,2932,2022/02/03,Image,microCT,TBD-Cage112-2932
